In [170]:
import pandas as pd
import numpy as np
import json
from yahoo_oauth import OAuth2
oauth = OAuth2(None, None, from_file='oauth.json')
if not oauth.token_is_valid():
    oauth.refresh_access_token()
oauth.oauth.base_url='https://fantasysports.yahooapis.com/fantasy/v2/'

[2018-09-10 09:46:13,781 DEBUG] [yahoo_oauth.yahoo_oauth.__init__] Checking 
[2018-09-10 09:46:13,782 DEBUG] [yahoo_oauth.yahoo_oauth.token_is_valid] ELAPSED TIME : 9027.172005176544
[2018-09-10 09:46:13,783 DEBUG] [yahoo_oauth.yahoo_oauth.token_is_valid] TOKEN HAS EXPIRED
[2018-09-10 09:46:13,783 DEBUG] [yahoo_oauth.yahoo_oauth.refresh_access_token] REFRESHING TOKEN
[2018-09-10 09:46:13,955 DEBUG] [yahoo_oauth.yahoo_oauth.token_is_valid] ELAPSED TIME : 0.17189717292785645
[2018-09-10 09:46:13,956 DEBUG] [yahoo_oauth.yahoo_oauth.token_is_valid] TOKEN IS STILL VALID


In [171]:
oauth.refresh_access_token()

[2018-09-10 09:46:16,887 DEBUG] [yahoo_oauth.yahoo_oauth.refresh_access_token] REFRESHING TOKEN


{'access_token': 'xxaXfACRolj.72Q9LwD9ikjWorTDU66dQ6HZ_uAotaV0GNdJbh4mDk3lzaDzzx8XKBmgskR5J5j9hI9tLc2AjWFlmI.GoPV7Skb3AHyPspHPFcaysaKDds.Le2bDvQMdGsP0Mqodno3HjXKAmcuByr7mAgg4xJJFUXEDVdvHqEU3i_JxMuYo.FElqf9WEQUqrv2.6.QXkE3mKwXo6Mvv11Htz9OQLD.3pVef_wJOKnkXTJBBH3JSR0qlX.qGOaykQ53hzAkkZ4BDpaD2Gav.FP390OlgbwhbPehmD3QV1PVfDphwFFZPrkcmdXU6D_FiiCdwJt7MBpy5w7xHu0jnS2dYap5rsv2b6ji0i6banZfXid8.fzUwkpcb6qWTmOGH9A0Cv58Yzyu8Qn74wZ1P16CGopUOQsmqDGaO._w_6Y698.FiFsyA4rz6bznr9HFc8kFItirPfmIzFXcL4Ttos9LNxkE7ZylrA7o2m41nQwk0_ttvi3Ok9mA9I9sqEBoWglE3mR4vBkMFI8T5tMdT53aUMoOfLPgmzm9K3Hrg9Qm_CzTMuMOqcrbd.q6NX91Bsh9YjT9amf3Xb6FOYRV9TPOYa72.UCdWBsave5gkXe9..pM9bsu9RDD618DQP4Evio_Msg_4B8w7W.kQ7FqDQYAxsZHM0VhQWZrR.wEEDFzR7acx73moXMXXVlTcDfQFpPz_6.1QrJsqRugYdibwaF.dFhh_bLFEJnLfkihXt_vkiw5dSHCb08UTXKZ_xB_fF_r_gkMQjnrAECeUGZFng7LrS_EM5CN6zn.gbc5kM5xz0.CEvOloC.XneVuzrup4CuTnmPVNfhZH25P8qUjWh5uLrZHGc.U1GKy_Pf.EdmcsYl3CS3if8815barPcB7LcI394E.EdIe17f_CU6kA73A0YTI4jqXAgiGfNiznzL0L8Rurv79iUPidDboMtA.iUKYNTdxk6AtA2ut8MUU3zs

In [172]:
from fantasy_sport import FantasySport
yfs = FantasySport(oauth, fmt='json')

In [173]:
#%%
#response = json.loads(yfs.get_leagues_teams(['353.l.38761']).content)
#league_string = '380.l.171315' #2019 pragma
league_string = '380.1.1528762' #2019 paid league
#league_string = '380.l.9544123' #test draft
#league_string = '380.l.171315' #2019 pragma
#league_string = '371.l.72059'#
#league_string = '359.l.698246'
ffl_dir = 'D:/workspace/yahoo-ffl'

In [ ]:
json.loads(oauth.session.get('games;seasons=2018', params={'format': 'json'}).content)

In [174]:
#%%
response = json.loads(oauth.session.get('league/'+league_string+'/teams', params={'format':'json'}).content)
teamnames = [x['team'][0][2]['name'] for k,x in response['fantasy_content']['league'][1]['teams'].items() if k not in ['count']]
teamkeys = [x['team'][0][0]['team_key'] for k,x in response['fantasy_content']['league'][1]['teams'].items() if k not in ['count']]

KeyError: 'fantasy_content'

In [176]:
json.loads(oauth.session.get('leagues', params={'format':'json'}).content)

{'error': {'description': 'missing required param: league_keys',
  'detail': '',
  'xml:lang': 'en-us',
  'yahoo:uri': '/fantasy/v2/leagues?format=json'}}

In [103]:
#%%

playersByKey = {} 
for start in range(1,2000,25):
    response = json.loads(oauth.session.get('leagues;league_keys='+league_string+'/players;start='+str(start)+"/stats", params={'format':'json'}).content)
    players = response['fantasy_content']['leagues']['0']['league'][1]['players']
    if(not players):
        break
    playersByKey.update({x['player'][0][0]['player_key']:{next(iter(d.keys())):next(iter(d.values())) for d in x['player'][0] if isinstance(d, dict)} for k,x in players.items() if k not in ['count']})

In [104]:
#%%
playerFrame = pd.DataFrame.from_dict(playersByKey, orient='index')
playerFrame['name'] = playerFrame.apply(lambda x:x['name']['full'], axis=1)
playerFrame['eligible_positions'] = playerFrame.apply(lambda x:[str(next(iter(d.values()))) for d in x['eligible_positions']], axis=1)
playerFrame['bye_weeks'] = playerFrame.apply(lambda x:next(iter(x['bye_weeks'].values())), axis=1)

In [105]:
#%%

playerStats = pd.concat ([pd.read_csv(ffl_dir + "/stats-2019.csv").assign(year=2019),
pd.read_csv(ffl_dir + "/stats-2018.csv").assign(year=2018),
pd.read_csv(ffl_dir + "/stats-2017.csv").assign(year=2017)
                         ],sort=False)
playerStats['pos'] = playerStats['position'].apply(lambda x:x[x.find(' - ')+3:] if x.find(' - ') >0 else x)
#playerStats['avgPts'] = playerStats['points']/playerStats['games']

In [177]:
#%%
json.loads(yfs._get('leagues;league_keys='+league_string+'/players;count=1').content)['fantasy_content']['leagues']['0']['league']

league_settings = json.loads(oauth.session.get('leagues;league_keys='+league_string+'/settings', params={'format': 'json'}).content)['fantasy_content']['leagues']['0']['league'][1]['settings'][0]
roster_positions = pd.DataFrame([x['roster_position'] for x in league_settings['roster_positions']])
stat_settings = pd.DataFrame([x['stat'] for x in league_settings['stat_categories']['stats']])
stat_settings = pd.merge(stat_settings, pd.DataFrame([x['stat'] for x in league_settings['stat_modifiers']['stats']]), on='stat_id')
team_count = int(league_settings['max_teams'])

[2018-09-10 09:49:46,353 DEBUG] [yahoo_oauth.yahoo_oauth.token_is_valid] ELAPSED TIME : 209.46524024009705
[2018-09-10 09:49:46,354 DEBUG] [yahoo_oauth.yahoo_oauth.token_is_valid] TOKEN IS STILL VALID


KeyError: 'fantasy_content'

In [107]:
roster_positions = roster_positions.assign(repl_rank=roster_positions['count']*team_count)
#roster_positions = roster_positions.set_index('position')

In [108]:
stat_map = {
    'passing_yds':'Pass Yds',
    'passing_tds':'Pass TD',
    'passing_int':'Int',
    'rushing_yds':'Rush Yds',
    'rushing_tds':'Rush TD',
    'receiving_rec':'Rec',
    'receiving_yds':'Rec Yds',
    'receiving_tds':'Rec TD',
    'fumbles':'Fum Lost'
}

In [109]:
playerStats.rename(index=str, columns=stat_map)[list(stat_map.values())].iloc[0]

Pass Yds    287.0
Pass TD       2.3
Int           0.6
Rush Yds     16.2
Rush TD       0.1
Rec           0.0
Rec Yds       0.0
Rec TD        0.0
Fum Lost      0.1
Name: 0, dtype: float64

In [110]:
stat_multiplier = pd.to_numeric(stat_settings.loc[stat_settings.position_type=='O'].set_index('display_name').value.loc[list(stat_map.values())])

In [111]:
stat_multiplier

display_name
Pass Yds    0.04
Pass TD     4.00
Int        -2.00
Rush Yds    0.10
Rush TD     6.00
Rec         1.00
Rec Yds     0.10
Rec TD      6.00
Fum Lost   -2.00
Name: value, dtype: float64

In [112]:
playerStats['points_real'] = playerStats.rename(index=str, columns=stat_map)[list(stat_map.values())].dot(stat_multiplier).reset_index(drop=True)
stats_mean = playerStats.groupby(['name','pos']).points_real.agg({'points_real':'mean','count':'count'}).reset_index()

C:\Users\Jeff\AppData\Local\conda\conda\envs\cryptotrading\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


In [113]:
stats_mean = stats_mean[stats_mean['count']>3]

In [114]:
#%%
response = json.loads(yfs._get('leagues;league_keys='+league_string+'/draftresults').content)
response = response['fantasy_content']['leagues']['0']['league'][1]['draft_results']
draftresults = pd.DataFrame([x['draft_result'] for k,x in response.items() if k not in ['count'] and 'player_key' in x['draft_result']])
draftresults = draftresults.assign(name=playerFrame.loc[draftresults.player_key].name.reset_index(drop=True))
draftresults

[2018-09-09 11:40:09,743 DEBUG] [yahoo_oauth.yahoo_oauth.token_is_valid] ELAPSED TIME : 57.743525981903076
[2018-09-09 11:40:09,744 DEBUG] [yahoo_oauth.yahoo_oauth.token_is_valid] TOKEN IS STILL VALID


,cost,pick,player_key,round,team_key,name
0,62,1,380.p.28398,1,380.l.171315.t.3,Todd Gurley II
1,32,2,380.p.5228,1,380.l.171315.t.3,Tom Brady
2,59,3,380.p.28474,1,380.l.171315.t.9,David Johnson
3,58,4,380.p.26671,1,380.l.171315.t.11,Le'Veon Bell
4,52,5,380.p.24793,1,380.l.171315.t.7,Julio Jones
5,41,6,380.p.24791,1,380.l.171315.t.9,A.J. Green
6,26,7,380.p.7200,1,380.l.171315.t.10,Aaron Rodgers
7,64,8,380.p.29238,1,380.l.171315.t.6,Ezekiel Elliott
8,65,9,380.p.24171,1,380.l.171315.t.12,Antonio Brown
9,54,10,380.p.30972,1,380.l.171315.t.3,Saquon Barkley


In [147]:
rep_rank_map = roster_positions.set_index('position').repl_rank.to_dict()
replacement={}
replacement['QB'] = stats_mean.loc[stats_mean.pos=='QB'].sort_values('points_real', ascending=False).reset_index(drop=True).iloc[rep_rank_map['QB']+1].points_real
#remaining players after filling position specific slots
wrt_players = stats_mean.loc[stats_mean.pos.isin(['WR', 'RB', 'TE'])].sort_values('points_real', ascending=False).reset_index(drop=True)
pos_players = wrt_players.iloc[rep_rank_map['W/R/T']+1:]

In [150]:
bench_players = []
for position in ['WR', 'RB', 'TE']:
    pos_players = remaining_players.loc[remaining_players.pos==position].sort_values('points_real', ascending=False).reset_index(drop=True)
    bench_players.append(pos_players.iloc[rep_rank_map[position]:])
    replacement[position] = remaining_players.iloc[rep_rank_map[position]].points_real
        
#stats_mean['vorp'] = stats_mean.apply(lambda x:x.points_real-replacement[x.pos], axis=1)
stats_mean['vorp'] = stats_mean.points_real-stats_mean.pos.replace(replacement)

In [154]:
#replacement.update(wrt_players.iloc[rep_rank_map['W/R/T']+2:].groupby('pos').points_real.max().to_dict())
replacement
rep_rank_map

{'BN': 60,
 'DEF': 12,
 'K': 12,
 'QB': 12,
 'RB': 24,
 'TE': 12,
 'W/R/T': 12,
 'WR': 36}

In [117]:
remaining_players = stats_mean#[~stats_mean.name.isin(list(draftresults.name))]
spent_val = 0#pd.to_numeric(draftresults.cost).sum()

In [118]:
playerStats[playerStats.name=='Taylor Gabriel']

,week,name,position,team,opp,bye_week,passing_yds,passing_tds,passing_int,rushing_att,...,receiving_rec,receiving_yds,receiving_tds,return_tds,twopt,fumbles,points,year,pos,points_real
786,8,Taylor Gabriel,WR,Atl,NYJ,NaN,0.0,0.0,0.0,1.3,...,3.4,51.6,0.3,0.0,0.0,0.0,9.94,2018,WR,16.0
891,9,Taylor Gabriel,WR,Atl,Car,NaN,0.0,0.0,0.0,1.2,...,3.2,49.0,0.3,0.0,0.0,0.0,9.45,2018,WR,17.0


In [151]:

vorp_val = (int(league_settings['max_teams']) * 200 - spent_val) / stats_mean[stats_mean.vorp>=0].vorp.sum()
stats_mean['val'] = stats_mean['vorp'] * vorp_val
remaining_players = stats_mean#[~stats_mean.name.isin(list(draftresults.name))]


In [156]:
stats_mean.sort_values('val', ascending=False).reset_index()[:50]

,index,name,pos,points_real,count,vorp,val
0,215,Todd Gurley II,RB,20.448750,16,7.400054,65.873367
1,12,Alvin Kamara,RB,19.482500,16,6.433804,57.272060
2,196,Saquon Barkley,RB,18.541875,16,5.493179,48.898860
3,18,Antonio Brown,WR,15.827708,48,5.252324,46.754825
4,59,DeAndre Hopkins,WR,15.605417,48,5.030032,44.776043
5,172,Odell Beckham Jr.,WR,15.184583,48,4.609199,41.029893
6,57,Davante Adams,WR,14.827813,32,4.252428,37.854012
7,4,Adam Thielen,WR,14.822188,32,4.246803,37.803939
8,121,Julio Jones,WR,14.729375,48,4.153990,36.977747
9,160,Michael Thomas,WR,14.641875,32,4.066490,36.198844


In [411]:
stats_mean[stats_mean.name=='Mike Evans']

,name,pos,points_real,count,vorp,vorp_wrt,val
161,Mike Evans,WR,14.428333,48,1.341377,1.121458,23.755589


In [412]:
remaining_players[stats_mean.pos=='WR'].sort_values('val', ascending=False)

,name,pos,points_real,count,vorp,vorp_wrt,val
18,Antonio Brown,WR,15.827708,48,2.740752,2.520833,48.538318
59,DeAndre Hopkins,WR,15.605417,48,2.518460,2.298542,44.601565
172,Odell Beckham Jr.,WR,15.184583,48,2.097627,1.877708,37.148668
57,Davante Adams,WR,14.827813,32,1.740856,1.520938,30.830308
4,Adam Thielen,WR,14.822188,32,1.735231,1.515312,30.730690
121,Julio Jones,WR,14.729375,48,1.642418,1.422500,29.086994
160,Michael Thomas,WR,14.641875,32,1.554918,1.335000,27.537381
208,Ted Ginn Jr.,WR,14.640000,4,1.553043,1.333125,27.504175
0,A.J. Green,WR,14.547708,48,1.460752,1.240833,25.869703
161,Mike Evans,WR,14.428333,48,1.341377,1.121458,23.755589


In [261]:
#%%
response = json.loads(yfs.get_teams_roster(teamkeys).content)
teams = {team['team'][0][2]['name']:[player['player'][0][2]['name']['full'] 
              for p,player in team['team'][1]['roster']['0']['players'].items() if p not in ['count']] 
              for t,team in response['fantasy_content']['teams'].items() if t not in ['count']}

[2018-09-05 22:23:40,142 DEBUG] [yahoo_oauth.yahoo_oauth.token_is_valid] ELAPSED TIME : 8.415428638458252
[2018-09-05 22:23:40,143 DEBUG] [yahoo_oauth.yahoo_oauth.token_is_valid] TOKEN IS STILL VALID


In [262]:
playerFrame

,player_key,player_id,name,editorial_player_key,editorial_team_key,editorial_team_full_name,editorial_team_abbr,bye_weeks,uniform_number,display_position,headshot,is_undroppable,position_type,eligible_positions,status,has_player_notes,has_recent_player_notes,injury_note
380.p.100001,380.p.100001,100001,Atlanta,nfl.p.100001,nfl.t.1,Atlanta Falcons,Atl,8,False,DEF,{'url': 'https://s.yimg.com/lq/i/us/sp/v/nfl/t...,0,DT,[DEF],NaN,NaN,NaN,NaN
380.p.100002,380.p.100002,100002,Buffalo,nfl.p.100002,nfl.t.2,Buffalo Bills,Buf,11,False,DEF,{'url': 'https://s.yimg.com/lq/i/us/sp/v/nfl/t...,0,DT,[DEF],NaN,NaN,NaN,NaN
380.p.100003,380.p.100003,100003,Chicago,nfl.p.100003,nfl.t.3,Chicago Bears,Chi,5,False,DEF,{'url': 'https://s.yimg.com/lq/i/us/sp/v/nfl/t...,0,DT,[DEF],NaN,NaN,NaN,NaN
380.p.100004,380.p.100004,100004,Cincinnati,nfl.p.100004,nfl.t.4,Cincinnati Bengals,Cin,9,False,DEF,{'url': 'https://s.yimg.com/lq/i/us/sp/v/nfl/t...,0,DT,[DEF],NaN,NaN,NaN,NaN
380.p.100005,380.p.100005,100005,Cleveland,nfl.p.100005,nfl.t.5,Cleveland Browns,Cle,11,False,DEF,{'url': 'https://s.yimg.com/lq/i/us/sp/v/nfl/t...,0,DT,[DEF],NaN,NaN,NaN,NaN
380.p.100006,380.p.100006,100006,Dallas,nfl.p.100006,nfl.t.6,Dallas Cowboys,Dal,8,False,DEF,{'url': 'https://s.yimg.com/lq/i/us/sp/v/nfl/t...,0,DT,[DEF],NaN,NaN,NaN,NaN
380.p.100007,380.p.100007,100007,Denver,nfl.p.100007,nfl.t.7,Denver Broncos,Den,10,False,DEF,{'url': 'https://s.yimg.com/lq/i/us/sp/v/nfl/t...,0,DT,[DEF],NaN,NaN,NaN,NaN
380.p.100008,380.p.100008,100008,Detroit,nfl.p.100008,nfl.t.8,Detroit Lions,Det,6,False,DEF,{'url': 'https://s.yimg.com/lq/i/us/sp/v/nfl/t...,0,DT,[DEF],NaN,NaN,NaN,NaN
380.p.100009,380.p.100009,100009,Green Bay,nfl.p.100009,nfl.t.9,Green Bay Packers,GB,7,False,DEF,{'url': 'https://s.yimg.com/lq/i/us/sp/v/nfl/t...,0,DT,[DEF],NaN,NaN,NaN,NaN
380.p.100010,380.p.100010,100010,Tennessee,nfl.p.100010,nfl.t.10,Tennessee Titans,Ten,8,False,DEF,{'url': 'https://s.yimg.com/lq/i/us/sp/v/nfl/t...,0,DT,[DEF],NaN,NaN,NaN,NaN
